<a href="https://colab.research.google.com/github/huiju-kim/strawberry_model/blob/main/strawberry_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
# 두 가지 카테고리 분류
# zzllrr Imager Geek 다운로드
# 수집하고 싶은 이미지 검색
# 우측 상단 아이콘 클릭 => GO 버튼 누른 후 화면 전환
# 필요 없는 이미지, 아이콘 파일 클릭해서 제거
# rename 버튼 클릭 후 prefix 버튼 클릭 이름 명명 규칙 수정
# 수정한 이미지 이름 preview 버튼 클릭 후 반영 확인
# save 누른 후 수집 대기 -> 수집 완료 후
# 해당 카테고리 폴더 생성 후 모두 저장
# 상위 폴더 생성 후 카테고리 폴더를 모두 넣기
# 상위 폴더 압축 -> our_dataset.zip


# 목표 성능 90%
# 현재 89.8% -> HT
# 현재 85%
# 성능을 올리기 위해서는
# 1. DATA
# 2. Model(Model 교체 및 layer의 수를 추가)
# 3. HT

# 주제 : 딸기와 썩은 딸기 분류

In [3]:
# drive에 압축파일 업로드 하고
# 아래 코드 실행
!unzip /content/drive/MyDrive/strawberry_data.zip

Archive:  /content/drive/MyDrive/strawberry_data.zip
replace strawberry_data/rotten_strawberry/rotten_strawberry-01.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-01.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-02.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-03.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-04.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-05.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-06.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-07.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-08.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-09.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-10.jpeg  
  inflating: strawberry_data/rotten_strawberry/rotten_strawberry-100.jpeg  
  inflating:

In [4]:
# 파일을 접근 -> 메모리에 파일을 올려야 -> 모든 파일의 경로가 필요
from glob import glob
mother_path = '/content/strawberry_data'
# dataset 폴더가 없는 상태에서 풀렸다면
# mother_path = '/content/
glob(mother_path + '/*/*')

['/content/strawberry_data/strawberry/strawberry-297.jpeg',
 '/content/strawberry_data/strawberry/strawberry-198.jpeg',
 '/content/strawberry_data/strawberry/strawberry-320.jpeg',
 '/content/strawberry_data/strawberry/strawberry-639.jpeg',
 '/content/strawberry_data/strawberry/strawberry-652.jpeg',
 '/content/strawberry_data/strawberry/strawberry-671.jpeg',
 '/content/strawberry_data/strawberry/strawberry-741.jpeg',
 '/content/strawberry_data/strawberry/strawberry-637.jpeg',
 '/content/strawberry_data/strawberry/strawberry-286.jpeg',
 '/content/strawberry_data/strawberry/strawberry-600.jpeg',
 '/content/strawberry_data/strawberry/strawberry-359.jpeg',
 '/content/strawberry_data/strawberry/strawberry-765.jpeg',
 '/content/strawberry_data/strawberry/strawberry-466.jpeg',
 '/content/strawberry_data/strawberry/strawberry-485.jpeg',
 '/content/strawberry_data/strawberry/strawberry-782.jpeg',
 '/content/strawberry_data/strawberry/strawberry-391.jpeg',
 '/content/strawberry_data/strawberry/st

In [5]:
import os
a = '/content'
b = 'test'
os.path.join(a,b) # = a+'/'+b 리눅스, 맥, window

'/content/test'

In [6]:
from PIL import Image
import numpy as np

In [7]:
folder_list = ['rotten_strawberry', 'strawberry']
train = []
label = []
for idx , folder_name in enumerate(folder_list):
    folder = os.path.join(mother_path, folder_name)
    all_files_path = glob(folder+'/*')

    for k,file_path in enumerate(all_files_path):
        # 파일 불러오기
        img = Image.open(file_path)
        # 이미지 파일 전처리
        img = img.resize((128,128), Image.BILINEAR) # 양선형 보간법 사용
        img = img.convert('RGB')
        np_img = np.array(img)
        train.append(np_img)
        label.append(idx)
        if k % 20 == 0:
          print(f'{k} / {len(all_files_path)}')

np_train = np.array(train)
np_label = np.array(label)



0 / 850
20 / 850
40 / 850
60 / 850
80 / 850
100 / 850
120 / 850
140 / 850
160 / 850
180 / 850
200 / 850
220 / 850
240 / 850
260 / 850
280 / 850
300 / 850
320 / 850
340 / 850
360 / 850
380 / 850
400 / 850
420 / 850
440 / 850
460 / 850
480 / 850
500 / 850
520 / 850
540 / 850
560 / 850
580 / 850
600 / 850
620 / 850
640 / 850
660 / 850
680 / 850
700 / 850
720 / 850
740 / 850
760 / 850
780 / 850
800 / 850
820 / 850
840 / 850
0 / 799
20 / 799
40 / 799
60 / 799
80 / 799
100 / 799
120 / 799
140 / 799
160 / 799
180 / 799
200 / 799
220 / 799
240 / 799
260 / 799
280 / 799
300 / 799
320 / 799
340 / 799
360 / 799
380 / 799
400 / 799
420 / 799
440 / 799
460 / 799
480 / 799
500 / 799
520 / 799
540 / 799
560 / 799
580 / 799
600 / 799
620 / 799
640 / 799
660 / 799
680 / 799
700 / 799
720 / 799
740 / 799
760 / 799
780 / 799


In [8]:
print(np.bincount(np_label))
print(np_train.shape)
#np.save()

[850 799]
(1649, 128, 128, 3)


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np_train, np_label, test_size=0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1319, 128, 128, 3)
(330, 128, 128, 3)
(1319,)
(330,)


## Normalize 를 하는 이유


*   통계 및 통계 응용에서 정규화는 다양한 의미를 가질 수 있습니다.
*   가장 단순한 경우에, 등급의 정규화는 종종 평균화하기 전에 다른 척도에서 측정된 값을 개념적으로 일반적인 척도로 조정하는 것을 의미합니다

In [10]:
# xy = (x_train,x_test,y_train,y_test)
# np.save('preprocessed_data.npy')

In [11]:
x_train = x_train / 255.
x_test = x_test / 255.

In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 하이퍼 파라미터 설정
learning_rate = 0.001
dropout_rate = 0.2

input_layer = layers.Input(shape=(128,128,3), name='start layer')
tower_1 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_1' )(input_layer)
tower_2 = layers.Conv2D(128 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_1')(input_layer)
tower_2 = layers.Conv2D(128 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2')(tower_2)
tower_3 = layers.Conv2D(256 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_1')(input_layer)
tower_3 = layers.Conv2D(256 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_2')(tower_3)
tower_4 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_4_1')(input_layer)
tower_4 = layers.Conv2D(512 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_4_2')(tower_4)
#
concat_layer = layers.concatenate([tower_1,tower_2,tower_3,tower_4],axis=3)
concat_layer = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer)
##
tower_2_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_2_1_1' )(concat_layer)
tower_2_2_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_2_1')(concat_layer)
tower_2_2_2 = layers.Conv2D(120 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2_2')(tower_2_2_1)
tower_2_3_1 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_1')(concat_layer)
tower_2_3_2 = layers.Conv2D(120 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_2')(tower_2_3_1)
tower_2_4_1 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_2_4_1')(concat_layer)
tower_2_4_2 = layers.Conv2D(120 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_4_2')(tower_2_4_1)
concat_layer_2 = layers.concatenate([tower_2_1,tower_2_2_2,tower_2_3_2,tower_2_4_2],
                                  axis=3)
concat_layer_2 = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer_2)
#
last_avg_pool = layers.AveragePooling2D(pool_size=(31,31) , strides=(1,1), padding='valid')(concat_layer_2)
flat_layer = layers.Flatten()(last_avg_pool)
output_layer = layers.Dense(1 ,activation='sigmoid')(flat_layer)

# 변경 포인트 input, lastavg_pool,output layer
model = models.Model(input_layer,output_layer)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start layer (InputLayer)    [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 tower_2_1 (Conv2D)          (None, 128, 128, 128)        512       ['start layer[0][0]']         
                                                                                                  
 tower_3_1 (Conv2D)          (None, 128, 128, 256)        1024      ['start layer[0][0]']         
                                                                                                  
 tower_4_1 (MaxPooling2D)    (None, 128, 128, 3)          0         ['start layer[0][0]']         
                                                                                            

In [20]:
!mkdir tensorboard_log

mkdir: cannot create directory ‘tensorboard_log’: File exists


In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])
tb = tf.keras.callbacks.TensorBoard(log_dir='/content/tensorboard_log',
                                    histogram_freq=1)
#
model.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          epochs=10,callbacks=[tb])

Epoch 1/10
42/42 [==============================] - 49s 1s/step - loss: 0.6881 - accuracy: 0.5792 - val_loss: 0.6672 - val_accuracy: 0.6697
Epoch 2/10
42/42 [==============================] - 45s 1s/step - loss: 0.6180 - accuracy: 0.6649 - val_loss: 0.6041 - val_accuracy: 0.6485
Epoch 3/10
42/42 [==============================] - 46s 1s/step - loss: 0.5802 - accuracy: 0.7324 - val_loss: 0.5347 - val_accuracy: 0.7606
Epoch 4/10
42/42 [==============================] - 47s 1s/step - loss: 0.5360 - accuracy: 0.7582 - val_loss: 0.5432 - val_accuracy: 0.7091
Epoch 5/10
42/42 [==============================] - 49s 1s/step - loss: 0.5158 - accuracy: 0.7582 - val_loss: 0.7041 - val_accuracy: 0.6515
Epoch 6/10
42/42 [==============================] - 49s 1s/step - loss: 0.5201 - accuracy: 0.7672 - val_loss: 0.4841 - val_accuracy: 0.7758
Epoch 7/10
42/42 [==============================] - 50s 1s/step - loss: 0.4825 - accuracy: 0.7824 - val_loss: 0.4606 - val_accuracy: 0.7697
Epoch 8/10
42/42 [==

In [22]:
from google.colab import files
f = files.upload()

Saving test1.jpg to test1.jpg


In [23]:
test_file_name = list(f.keys())[0]

In [24]:
img = Image.open(test_file_name)
img = img.resize((128,128))
img = img.convert('RGB')
data = np.asarray(img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor / 255
test_image_tensor.shape

(1, 128, 128, 3)

In [25]:
result = model.predict(test_image_tensor)
if result[0][0] > 0.5:
    print(f'{folder_list[1]} 일 확률이 {result[0][0] * 100} % 입니다')
else:
    print(f'{folder_list[0]} 일 확률이 {(1 - result[0][0])* 100} % 입니다')


1/1 [==============================] - 0s 184ms/step
strawberry 일 확률이 60.00334620475769 % 입니다
